In this notebook, I will be pulling imagery from the year before a fire in the Great Smoky Mountains and after the fire. We will be comparing NDVI and NBR along with pulling species occurrence records from before and after the fire and see how it overlaps with occurrence records.

#### Declare ROI

In [18]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="ee-krle4401")

site_center = ee.Geometry.Point([-83.5, 35.7])

In [19]:
import geemap

#### Pull NEON Imagery from GEE

#### NDVI Code

In [ ]:
# Load AOP Image Collection
sdr_col = ee.ImageCollection('projects/neon-prod-earthengine/assets/HSI_REFL/001')

# Retrieve all available image IDs
image_ids = sdr_col.aggregate_array("system:index").getInfo()
print("🔍 Available AOP Image IDs:", image_ids)

# **Function to Filter Images by Multiple Years & Domains**
def filter_aop_images(years, domains):
    """
    Filters the AOP image collection based on a list of years and domains.
    
    Parameters:
        years (list): List of years (e.g., ["2016", "2017"]).
        domains (list): List of 4-letter site codes (e.g., ["GRSM", "HARV"]).
    
    Returns:
        Dictionary with (year, domain) as key and list of matching image IDs.
    """
    filtered_results = {}

    for year in years:
        for domain in domains:
            matching_ids = [img_id for img_id in image_ids if year in img_id and domain in img_id]
            if matching_ids:
                filtered_results[(year, domain)] = matching_ids

    return filtered_results

# **User Inputs**
years_input = ["2016", "2017"]  # Add multiple years here
domains_input = ["GRSM"]  # Add multiple domains here

# Get filtered images
filtered_images = filter_aop_images(years_input, domains_input)

if not filtered_images:
    print(f"No AOP images found for the selected years and domains.")
else:
    print(f"Found AOP images: {filtered_images}")

# NDVI Visualization Parameters
ndvi_vis_params = {
    'min': -1, 'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Create a geemap Map instance
Map = geemap.Map()
Map.centerObject(site_center, 10)

# NBR Visualization Parameters
nbr_vis_params = {
    'min': -1, 'max': 1,
    'palette': ['white', 'yellow', 'red', 'black']  # typical burn severity color ramp
}

# Function to Compute NBR
def addNBRBands(image):
    nbr = image.normalizedDifference(['B097', 'B220']).rename('NBR')
    return image.addBands(nbr).set({'Sensor': 'AOP'})


# Loop through filtered images and compute NDVI + NBR
for (year, domain), image_ids in filtered_images.items():
    for image_id in image_ids:
        # Retrieve the image by its system:index
        aop_image = sdr_col.filter(ee.Filter.eq("system:index", image_id)).first()

        # Function to Compute NDVI
        def addAOPBands(image):
            ndvi = image.normalizedDifference(['B097', 'B055']).rename('NDVI')
            return image.addBands(ndvi).set({'Sensor': 'AOP'})

        # Compute NDVI
        if aop_image:
            aop_ndvi = addAOPBands(aop_image).select('NDVI')
            print(f"NDVI computed for AOP image: {image_id}")

            # Add layer with proper naming
            Map.addLayer(aop_ndvi, ndvi_vis_params, f'AOP NDVI ({image_id})')

# Save the final interactive HTML map with all selected images
html_filename = f"AOP_NDVI_{'_'.join(years_input)}_{'_'.join(domains_input)}.html"
Map.to_html(filename=html_filename)

print(f"NDVI visualization saved: {html_filename}")


🔍 Available AOP Image IDs: ['2013_CPER_1', '2014_HARV_2', '2014_JERC_1', '2015_MLBS_1', '2015_TALL_1', '2016_CLBJ_1', '2016_GRSM_2', '2016_HARV_3', '2016_JERC_2', '2016_SERC_1', '2016_TALL_2', '2017_CLBJ_2', '2017_CPER_3', '2017_GRSM_3', '2017_HARV_4', '2017_JERC_3', '2017_MLBS_2', '2017_OAES_2', '2017_SERC_3', '2017_SRER_1', '2017_TALL_3', '2018_GUAN_1', '2018_HARV_5', '2018_JERC_4', '2018_MCRA_1', '2018_MLBS_3', '2018_OAES_3', '2018_SRER_2', '2018_TALL_4', '2019_CLBJ_4', '2019_HARV_6', '2019_HEAL_3', '2019_JERC_5', '2019_JORN_3', '2019_NIWO_3', '2019_OAES_4', '2019_SERC_4', '2019_SOAP_4', '2019_SRER_3', '2019_TALL_5', '2020_CPER_5', '2020_CPER_7', '2020_NIWO_4', '2020_RMNP_3', '2020_YELL_3', '2021_ABBY_4', '2021_BONA_4', '2021_CLBJ_5', '2021_CPER_8', '2021_HEAL_4', '2021_JERC_6', '2021_JORN_4', '2021_MCRA_2', '2021_OAES_5', '2021_OSBS_6', '2021_SERC_5', '2021_SJER_5', '2021_SOAP_5', '2021_SRER_4', '2021_TALL_6']
Found AOP images: {('2016', 'GRSM'): ['2016_GRSM_2'], ('2017', 'GRSM'): 

#### NBR Code

In [5]:
# NBR Visualization Parameters
nbr_vis_params = {
    'min': -1, 'max': 1,
    'palette': ['white', 'yellow', 'red', 'black']  # typical burn severity color ramp
}

# Function to Compute NBR
def addNBRBands(image):
    nbr = image.normalizedDifference(['B097', 'B220']).rename('NBR')
    return image.addBands(nbr).set({'Sensor': 'AOP'})

# Create a geemap Map instance for NBR visualization
NBR_Map = geemap.Map()
NBR_Map.centerObject(site_center, 10)

# Loop through filtered images and compute NBR
for (year, domain), image_ids in filtered_images.items():
    for image_id in image_ids:
        # Retrieve the image by its system:index
        aop_image = sdr_col.filter(ee.Filter.eq("system:index", image_id)).first()

        # Compute NBR
        if aop_image:
            aop_nbr = addNBRBands(aop_image).select('NBR')
            print(f"NBR computed for AOP image: {image_id}")

            # Add layer with proper naming
            NBR_Map.addLayer(aop_nbr, nbr_vis_params, f'AOP NBR ({image_id})')

# Save the final interactive HTML map for NBR
html_nbr_filename = f"AOP_NBR_{'_'.join(years_input)}_{'_'.join(domains_input)}.html"
NBR_Map.to_html(filename=html_nbr_filename)

print(f"NBR visualization saved: {html_nbr_filename}")

NBR computed for AOP image: 2016_GRSM_2
NBR computed for AOP image: 2017_GRSM_3
NBR visualization saved: AOP_NBR_2016_2017_GRSM.html


#### Pull NBR and NDVI together as layers in a single map

In [6]:
# Load AOP Image Collection
sdr_col = ee.ImageCollection('projects/neon-prod-earthengine/assets/HSI_REFL/001')

# Retrieve all available image IDs
image_ids = sdr_col.aggregate_array("system:index").getInfo()
print("🔍 Available AOP Image IDs:", image_ids)

# Function to Filter Images by Multiple Years & Domains
def filter_aop_images(years, domains):
    filtered_results = {}

    for year in years:
        for domain in domains:
            matching_ids = [img_id for img_id in image_ids if year in img_id and domain in img_id]
            if matching_ids:
                filtered_results[(year, domain)] = matching_ids

    return filtered_results

# User Inputs
years_input = ["2016", "2017"]  # Add multiple years here
domains_input = ["GRSM"]  # Add multiple domains here

# Get filtered images
filtered_images = filter_aop_images(years_input, domains_input)

if not filtered_images:
    print(f"No AOP images found for the selected years and domains.")
else:
    print(f"Found AOP images: {filtered_images}")

# NDVI Visualization Parameters
ndvi_vis_params = {
    'min': -1, 'max': 1,
    'palette': ['blue', 'white', 'green']
}

# NBR Visualization Parameters
nbr_vis_params = {
    'min': -1, 'max': 1,
    'palette': ['white', 'yellow', 'red', 'black']
}

# Function to Compute NDVI and NBR
def addNDVI_NBR_Bands(image):
    ndvi = image.normalizedDifference(['B097', 'B055']).rename('NDVI')
    nbr = image.normalizedDifference(['B097', 'B220']).rename('NBR')
    return image.addBands([ndvi, nbr]).set({'Sensor': 'AOP'})

# Create a geemap Map instance
Map = geemap.Map()
Map.centerObject(site_center, 10)

# Loop through filtered images and compute NDVI + NBR
for (year, domain), image_ids in filtered_images.items():
    for image_id in image_ids:
        aop_image = sdr_col.filter(ee.Filter.eq("system:index", image_id)).first()

        if aop_image:
            aop_indices = addNDVI_NBR_Bands(aop_image)
            aop_ndvi = aop_indices.select('NDVI')
            aop_nbr = aop_indices.select('NBR')

            print(f"NDVI and NBR computed for AOP image: {image_id}")

            # Add layers with proper naming
            Map.addLayer(aop_ndvi, ndvi_vis_params, f'AOP NDVI ({image_id})')
            Map.addLayer(aop_nbr, nbr_vis_params, f'AOP NBR ({image_id})')

# Save the final interactive HTML map with all selected images
html_filename = f"AOP_NDVI_NBR_{'_'.join(years_input)}_{'_'.join(domains_input)}.html"
Map.to_html(filename=html_filename)

print(f"NDVI and NBR visualization saved: {html_filename}")


🔍 Available AOP Image IDs: ['2013_CPER_1', '2014_HARV_2', '2014_JERC_1', '2015_MLBS_1', '2015_TALL_1', '2016_CLBJ_1', '2016_GRSM_2', '2016_HARV_3', '2016_JERC_2', '2016_SERC_1', '2016_TALL_2', '2017_CLBJ_2', '2017_CPER_3', '2017_GRSM_3', '2017_HARV_4', '2017_JERC_3', '2017_MLBS_2', '2017_OAES_2', '2017_SERC_3', '2017_SRER_1', '2017_TALL_3', '2018_GUAN_1', '2018_HARV_5', '2018_JERC_4', '2018_MCRA_1', '2018_MLBS_3', '2018_OAES_3', '2018_SRER_2', '2018_TALL_4', '2019_CLBJ_4', '2019_HARV_6', '2019_HEAL_3', '2019_JERC_5', '2019_JORN_3', '2019_NIWO_3', '2019_OAES_4', '2019_SERC_4', '2019_SOAP_4', '2019_SRER_3', '2019_TALL_5', '2020_CPER_5', '2020_CPER_7', '2020_NIWO_4', '2020_RMNP_3', '2020_YELL_3', '2021_ABBY_4', '2021_BONA_4', '2021_CLBJ_5', '2021_CPER_8', '2021_HEAL_4', '2021_JERC_6', '2021_JORN_4', '2021_MCRA_2', '2021_OAES_5', '2021_OSBS_6', '2021_SERC_5', '2021_SJER_5', '2021_SOAP_5', '2021_SRER_4', '2021_TALL_6']
Found AOP images: {('2016', 'GRSM'): ['2016_GRSM_2'], ('2017', 'GRSM'): 

#### Pull Records from GBIF

In [14]:
from pygbif import occurrences as gbif_occ
import geopandas as gpd
from shapely.geometry import Point, Polygon

latitude, longitude = 35.6118, -83.4895
bbox_size_deg = 0.09  # Approx. 10 km (~0.09 degrees latitude/longitude)
bounding_box_coords = [
    (longitude - bbox_size_deg, latitude - bbox_size_deg),  # Bottom-left
    (longitude + bbox_size_deg, latitude - bbox_size_deg),  # Bottom-right
    (longitude + bbox_size_deg, latitude + bbox_size_deg),  # Top-right
    (longitude - bbox_size_deg, latitude + bbox_size_deg),  # Top-left
    (longitude - bbox_size_deg, latitude - bbox_size_deg),  # Close the polygon
]

# Create a Polygon object and convert it to WKT
bounding_polygon = Polygon(bounding_box_coords)
polygon_wkt = bounding_polygon.wkt  # Correctly formatted for GBIF

# GBIF query using centroid and radius
occurrences = gbif_occ.search(
    taxonKey=6,  # plants
    geometry= polygon_wkt,
    hasCoordinate=True,
    limit=300
)

# Extract and clean records from GBIF response
records = []
for occ in occurrences.get("results", []):
    try:
        lat = float(occ["decimalLatitude"])
        lon = float(occ["decimalLongitude"])

        records.append({
            "species": occ.get("species", "Unknown"),
            "latitude": lat,
            "longitude": lon,
        })
    except (KeyError, TypeError, ValueError):
        continue

# Create GeoDataFrame
geo_df = gpd.GeoDataFrame(
    records,
    geometry=[Point(xy) for xy in zip(
        [rec["longitude"] for rec in records],
        [rec["latitude"] for rec in records]
    )],
    crs="EPSG:4326"
)

# Save GeoDataFrame as GeoJSON
geo_df.to_file("gbif_gsmnp_centroid.geojson", driver="GeoJSON")

# Display first few records
print(geo_df.head())

INFO:Created 300 records


                 species   latitude  longitude                    geometry
0       Mitchella repens  35.643423 -83.575888  POINT (-83.57589 35.64342)
1       Carex fraseriana  35.686088 -83.419037  POINT (-83.41904 35.68609)
2      Huperzia lucidula  35.671424 -83.522858  POINT (-83.52286 35.67142)
3      Huperzia lucidula  35.663077 -83.452427  POINT (-83.45243 35.66308)
4  Platanus occidentalis  35.686755 -83.501092  POINT (-83.50109 35.68676)


#### Workflow Combined

In [17]:
# NEON and GBIF Integration for GSMNP (2016-2017)

import geemap
import ee
from pygbif import occurrences as gbif_occ
import geopandas as gpd
from shapely.geometry import Point, Polygon

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="ee-krle4401")

# GSMNP centroid
latitude, longitude = 35.6118, -83.4895
bbox_size_deg = 0.09  # Approx. 10 km (~0.09 degrees latitude/longitude)

# Create bounding box
bounding_box_coords = [
    (longitude - bbox_size_deg, latitude - bbox_size_deg),  # Bottom-left
    (longitude + bbox_size_deg, latitude - bbox_size_deg),  # Bottom-right
    (longitude + bbox_size_deg, latitude + bbox_size_deg),  # Top-right
    (longitude - bbox_size_deg, latitude + bbox_size_deg),  # Top-left
    (longitude - bbox_size_deg, latitude - bbox_size_deg),  # Close polygon
]

bounding_polygon = Polygon(bounding_box_coords)
polygon_wkt = bounding_polygon.wkt

# Query GBIF for 2016-2017 plant records within bounding polygon
records = []
for year in [2016, 2017]:
    occurrences = gbif_occ.search(
        taxonKey=6,  # plants
        geometry=polygon_wkt,
        year=year,
        hasCoordinate=True,
        limit=300
    )

    for occ in occurrences.get("results", []):
        try:
            lat = float(occ["decimalLatitude"])
            lon = float(occ["decimalLongitude"])

            records.append({
                "species": occ.get("species", "Unknown"),
                "latitude": lat,
                "longitude": lon,
                "year": year
            })
        except (KeyError, TypeError, ValueError):
            continue

# Convert GBIF records to GeoDataFrame
gbif_gdf = gpd.GeoDataFrame(
    records,
    geometry=gpd.points_from_xy([r['longitude'] for r in records], [r['latitude'] for r in records]),
    crs="EPSG:4326"
)

# Save GBIF records as GeoJSON
gbif_gdf.to_file("gbif_gsmnp_2016_2017.geojson", driver="GeoJSON")

# NEON imagery setup
sdr_col = ee.ImageCollection('projects/neon-prod-earthengine/assets/HSI_REFL/001')
image_ids = sdr_col.aggregate_array("system:index").getInfo()

# Filter NEON images
filtered_images = {year: [img_id for img_id in image_ids if str(year) in img_id and "GRSM" in img_id]
                   for year in [2016, 2017]}

# NDVI & NBR visualization parameters
ndvi_vis_params = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
nbr_vis_params = {'min': -1, 'max': 1, 'palette': ['white', 'yellow', 'red', 'black']}

# NDVI and NBR computation function
def addNDVI_NBR(image):
    ndvi = image.normalizedDifference(['B097', 'B055']).rename('NDVI')
    nbr = image.normalizedDifference(['B097', 'B220']).rename('NBR')
    return image.addBands([ndvi, nbr]).set({'Sensor': 'AOP'})

# Initialize geemap
Map = geemap.Map()
Map.centerObject(ee.Geometry.Point(longitude, latitude), 12)

# Plot NEON NDVI and NBR
for year, ids in filtered_images.items():
    for image_id in ids:
        image = sdr_col.filter(ee.Filter.eq("system:index", image_id)).first()
        if image:
            image_indices = addNDVI_NBR(image)
            Map.addLayer(image_indices.select('NDVI'), ndvi_vis_params, f"NDVI {image_id}")
            Map.addLayer(image_indices.select('NBR'), nbr_vis_params, f"NBR {image_id}")

# Plot GBIF records by year separately
for year in [2016, 2017]:
    year_gdf = gbif_gdf[gbif_gdf['year'] == year]
    Map.add_gdf(year_gdf, layer_name=f"GBIF Plants ({year})")

# Save combined interactive map
html_filename = "GSMNP_NEON_GBIF_2016_2017.html"
Map.to_html(filename=html_filename)

print(f"Combined NEON + GBIF map saved: {html_filename}")

INFO:Created 600 records


Combined NEON + GBIF map saved: GSMNP_NEON_GBIF_2016_2017.html
